In [ ]:
# install sp
import sys
!{sys.executable} -m pip install -U spacy

In [1]:
import spacy
import random

In [ ]:
nlp = spacy.load('en_core_web_sm')

## Train the spacy entity recognizition model.

In [ ]:
table_names = ['employee', 'organization', 'department']
column_names = ['id', 'name', 'email', 'address', 'phone_number', 'date_of_birth']

In [ ]:
all_input_vectors = table_names + column_names

In [2]:
train_data = [
    ("get list of employees", {
        'entities': [(12, 20, 'Table')]
    }),
    ("get list of employees with name Nilepta", {
        'entities': [(12, 20, 'Table'), (27, 31, 'Column'), (32, 39, 'Value')]
    }),
    ("search employees with name Nilepta", {
        'entities': [(7, 16, 'Table'), (22, 26, 'Column')]
    }),
    ("find list of employees who live in Washington state", {
        'entities': [(13, 22, 'Table'), (46, 51, 'Column'), (35, 45, 'Value')]
    }),
    ("get details of all employees from state Washington", {
        'entities': [(19, 28, 'Table'), (34, 39, 'Column'), (40, 50, 'Value')]
    }),
    ("find employee with phone number as 2345678901", {
        'entities': [(5, 13, 'Table'), (19, 24, 'Column'), (35, 45, 'Value')]
    }),
    ("get list of employees from Washington state", {
        'entities': [(12, 21, 'Table'), (38, 43, 'Column'), (27, 37, 'Value')]
    }),
    ("find employees whose email is qwerty@gmail.com", {
     'entities': [(5, 14, 'Table'), (21, 26, 'Column'), (30, 46, 'Value')]
    }),
    ("get employee whose email is asdfg@gmail.com", {
        'entities': [(4, 12, 'Table'), (19, 24, 'Column'), (28, 43, 'Value')]
    })
    ]

In [3]:
# nlp create model.
nlp = spacy.blank('en')

In [4]:
ner = nlp.create_pipe('ner')

In [5]:
nlp.add_pipe(ner)

In [6]:
# add labelled data to the model.
for _, annotations in train_data:
    for ent in annotations.get('entities'):
        ner.add_label(ent[2])

In [7]:
other_pipes = [pipe for pipe in nlp.pipe_names if pipe != 'ner']

In [8]:
n_iter = 20

In [9]:
with nlp.disable_pipes(*other_pipes):  # only train NER
    optimizer = nlp.begin_training()
    for itn in range(n_iter):
        random.shuffle(train_data)
        losses = {}
        for text, annotations in train_data:
            nlp.update([text], [annotations], sgd=optimizer, drop=0.35, losses=losses)
        print(losses)

{'ner': 48.27955485880375}
{'ner': 33.514296382665634}
{'ner': 29.562335779162822}
{'ner': 24.430849429318187}
{'ner': 16.95822232779574}
{'ner': 10.414102137197915}
{'ner': 4.1169833215617135}
{'ner': 4.353136730369077}
{'ner': 2.303695996729147}
{'ner': 1.4491652798667907}
{'ner': 2.186035048780056}
{'ner': 2.3461464835971753}
{'ner': 1.6844050177763976}
{'ner': 0.15422280577189593}
{'ner': 1.9895320629452442}
{'ner': 1.9472885652570562}
{'ner': 1.9875506246443473}
{'ner': 2.4948479159302046}
{'ner': 1.989930288355877}
{'ner': 0.09252979656753979}


In [11]:
# testing the model.
test_text = "get employee with email as rohan@gmail.com"
doc = nlp(test_text)
print("Entities in '%s'" % test_text)
for i, ent in enumerate(doc.ents):
    print("Entity number %s is %s in text: '%s'" % (i, ent.label_, ent.text))

Entities in 'get employee with email as rohan@gmail.com'
Entity number 0 is Table in text: 'employee'
Entity number 1 is Column in text: 'email'
Entity number 2 is Value in text: 'rohan@gmail.com'


In [12]:
ner.to_disk('/mnt/azmnt/code/Users/Congitveservice_NoSQL/NLP/ner-model-1')